In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/augmented-forest-segmentation/meta_data.csv
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/meta_data.csv
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/626208_sat_04.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/15573_sat_26.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/749523_sat_70.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/830444_sat_35.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/340898_sat_47.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/563092_sat_47.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/119079_sat_71.jpg
/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images/296279_sat_35.jpg
/kaggle/input/augmented-for

In [29]:
import tensorflow as tf
import matplotlib.pyplot as plt
ROOT_DIR = "/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented"

In [36]:
import imageio

images = os.listdir("/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/images")
masks = os.listdir("/kaggle/input/augmented-forest-segmentation/Forest Segmented/Forest Segmented/masks")

masks[0]

'15573_mask_48.jpg'

In [3]:
import cv2

_num = 103

fig, arr = plt.subplots(1, 2, sharey=True, figsize=(14,10))

image = cv2.imread(os.path.join(ROOT_DIR, "images", images[_num]))
img_arr = image.astype(np.float32)/255.0

mask = cv2.imread(os.path.join(ROOT_DIR, "masks", masks[_num]))
mask_arr = mask.astype(np.float32)/255.0

arr[0].imshow(img_arr)
arr[0].set_title("Image")
arr[1].imshow(mask_arr)
arr[1].set_title("Mask")

NameError: name 'plt' is not defined

In [1]:
def get_image_arr(image_path, mask_path):
    image = cv2.imread(image_path)
    img_arr = image.astype(np.float32)/255.0
    
    mask = cv2.imread(mask_path)
    mask_arr = mask.astype(np.float32)/255.0
    return image, mask

In [2]:
def conv_block(inputs=None, filters=32, dropout=0, max_pooling=False):
    
    conv = tf.keras.layers.Conv2D(filters, kernel_size=3, 
                                  activation="relu", padding="same", 
                                  kernel_initializer="he_normal")(input)
    conv = tf.keras.layers.Conv2D(filters, kernel_size=3, 
                                  activation="relu", padding="same", 
                                  kernel_initializer="he_normal")(conv)
    
    if dropout > 0:
        conv = tf.keras.layers.Dropout(dropout)(conv)
    
    if max_pooling:
        layer_new = tf.keras.layers.MaxPooling2D()(conv)
    else:
        layer_new = conv
    
    return layer_new, conv